# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

!jupyter nbextension enable --py gmaps
import gmaps.datasets

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("output_cities.csv")

weather_df.head()



,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flin flon,54.77,-101.86,55.40,66,20,5.82,CA,1590175860
1,1,georgetown,5.41,100.34,80.60,88,40,6.93,MY,1590175721
2,2,ushuaia,-54.80,-68.30,53.60,40,75,8.05,AR,1590175646
3,3,puerto ayora,-0.74,-90.35,84.20,62,20,16.11,EC,1590175860
4,4,hun,29.13,15.95,71.64,56,12,17.74,LY,1590175860


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) # Fill in with your API key
locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']
#to have the maps centered and zoomed. 
algeria_coordinates = (46.45, 3.2)
fig = gmaps.figure(center=algeria_coordinates,zoom_level=2)

fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
print(weather_df['Max Temp (F)'].max())

100.35


* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [6]:
# temp range
clean_df = weather_df.loc[(weather_df["Max Temp (F)"] > 70) & (weather_df["Max Temp (F)"] < 80)]
#wind speed
clean_df = clean_df.loc[clean_df["Wind Speed"] < 10]
#cloudiness
clean_df = clean_df.loc[clean_df["Cloudiness"] == 0]
clean_df = clean_df.dropna()
clean_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
217,217,sao filipe,14.90,-24.50,75.47,70,0,6.69,CV,1590175889
221,221,namibe,-15.20,12.15,76.60,59,0,7.36,AO,1590175890
269,269,komsomolskiy,40.43,71.72,71.60,43,0,6.93,UZ,1590175894
357,357,miandrivazo,-19.52,45.47,70.32,56,0,1.41,MG,1590175901
369,369,caravelas,-17.71,-39.25,75.87,82,0,4.70,BR,1590175902
396,396,dawlatabad,36.41,64.91,75.61,23,0,1.03,AF,1590175904
407,407,kosonsoy,41.26,71.55,71.60,43,0,6.93,UZ,1590175905
427,427,ambilobe,-13.20,49.05,70.92,80,0,1.99,MG,1590175907
511,511,maragogi,-9.01,-35.22,79.59,85,0,7.47,BR,1590175868


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

 Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

In [7]:
hotel_df = clean_df.copy()
#hotel_df

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
x=0
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city=row["City"]
    x=x+1
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
       
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        #hotelname=results[0]['name']
        print(f"{x}.No hotels within 5000 meters of {city}.")
        

In [8]:
hotel_df1 =hotel_df.dropna()
hotel_df1.to_csv('hotel_df.csv')


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df1.iterrows()]
locations = hotel_df1[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']
algeria_coordinates = (46.45, 3.2)
fig1 = gmaps.figure(center=algeria_coordinates,zoom_level=1)
fig1.add_layer(markers)
fig1.add_layer(hotel_layer)
fig1.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig1
# Display figure


Figure(layout=FigureLayout(height='420px'))